In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.compute_overlap_piou import compute_overlap
from utils.anchors import *

In [2]:
train_file = '/datasets/dataset/coco2017/annotations/instances_train2017_obb_cat_dog.csv'
val_file   = '/datasets/dataset/coco2017/annotations/instances_val2017_obb_cat_dog.csv'

In [3]:
# load annotations
df = pd.concat([pd.read_csv(train_file), pd.read_csv(val_file)])
df

,file_name,x1,y1,x2,y2,label,angle
0,000000574769.jpg,240.629295,142.158028,363.466713,264.995445,cat,2.015813
1,000000173350.jpg,172.734337,281.337975,381.152016,489.755653,dog,0.237836
2,000000173350.jpg,172.734337,281.337975,381.152016,489.755653,dog,0.237836
3,000000565797.jpg,82.800461,149.217102,499.131302,343.890564,cat,0.945045
4,000000241876.jpg,5.025818,-9.031616,446.586182,432.528748,cat,0.862608
...,...,...,...,...,...,...,...
411,000000360943.jpg,200.491947,192.809031,283.512508,225.057791,cat,1.604117
412,000000377000.jpg,-54.000000,84.000000,362.000000,392.000000,cat,3.141593
413,000000572388.jpg,-95.529373,96.592590,366.716019,366.216064,cat,3.136231
414,000000198641.jpg,158.888062,181.367126,450.940247,473.419312,cat,0.802940


In [4]:
ann = np.array(df[['x1', 'y1', 'x2', 'y2', 'angle']])
ann.shape, ann

((10542, 5),
 array([[ 2.40629295e+02,  1.42158028e+02,  3.63466713e+02,
          2.64995445e+02,  2.01581254e+00],
        [ 1.72734337e+02,  2.81337975e+02,  3.81152016e+02,
          4.89755653e+02,  2.37835931e-01],
        [ 1.72734337e+02,  2.81337975e+02,  3.81152016e+02,
          4.89755653e+02,  2.37835931e-01],
        ...,
        [-9.55293732e+01,  9.65925903e+01,  3.66716019e+02,
          3.66216064e+02,  3.13623078e+00],
        [ 1.58888062e+02,  1.81367126e+02,  4.50940247e+02,
          4.73419312e+02,  8.02940272e-01],
        [-2.53677521e+01,  9.62223969e+01,  4.86889755e+02,
          3.54876907e+02,  1.87940307e+00]]))

In [5]:
df['sizes'] = np.sqrt((df['x2'] - df['x1'])*(df['y2'] - df['y1']))
df['ratio'] = (df['y2'] - df['y1'])/(df['x2'] - df['x1'])
df

,file_name,x1,y1,x2,y2,label,angle,sizes,ratio
0,000000574769.jpg,240.629295,142.158028,363.466713,264.995445,cat,2.015813,122.837418,1.000000
1,000000173350.jpg,172.734337,281.337975,381.152016,489.755653,dog,0.237836,208.417679,1.000000
2,000000173350.jpg,172.734337,281.337975,381.152016,489.755653,dog,0.237836,208.417679,1.000000
3,000000565797.jpg,82.800461,149.217102,499.131302,343.890564,cat,0.945045,284.690299,0.467593
4,000000241876.jpg,5.025818,-9.031616,446.586182,432.528748,cat,0.862608,441.560364,1.000000
...,...,...,...,...,...,...,...,...,...
411,000000360943.jpg,200.491947,192.809031,283.512508,225.057791,cat,1.604117,51.742731,0.388443
412,000000377000.jpg,-54.000000,84.000000,362.000000,392.000000,cat,3.141593,357.949717,0.740385
413,000000572388.jpg,-95.529373,96.592590,366.716019,366.216064,cat,3.136231,353.032872,0.583291
414,000000198641.jpg,158.888062,181.367126,450.940247,473.419312,cat,0.802940,292.052185,1.000000


In [6]:
_, bins = np.histogram(df['ratio'], bins=3)
bins

array([0.        , 0.33333333, 0.66666667, 1.        ])

In [7]:
_, bins = np.histogram(df['sizes'], bins=5)
bins

array([  0.        , 133.41347656, 266.82695312, 400.24042969,
       533.65390625, 667.06738281])

In [8]:
df['scale'] = df['sizes']/534
df

,file_name,x1,y1,x2,y2,label,angle,sizes,ratio,scale
0,000000574769.jpg,240.629295,142.158028,363.466713,264.995445,cat,2.015813,122.837418,1.000000,0.230033
1,000000173350.jpg,172.734337,281.337975,381.152016,489.755653,dog,0.237836,208.417679,1.000000,0.390295
2,000000173350.jpg,172.734337,281.337975,381.152016,489.755653,dog,0.237836,208.417679,1.000000,0.390295
3,000000565797.jpg,82.800461,149.217102,499.131302,343.890564,cat,0.945045,284.690299,0.467593,0.533128
4,000000241876.jpg,5.025818,-9.031616,446.586182,432.528748,cat,0.862608,441.560364,1.000000,0.826892
...,...,...,...,...,...,...,...,...,...,...
411,000000360943.jpg,200.491947,192.809031,283.512508,225.057791,cat,1.604117,51.742731,0.388443,0.096896
412,000000377000.jpg,-54.000000,84.000000,362.000000,392.000000,cat,3.141593,357.949717,0.740385,0.670318
413,000000572388.jpg,-95.529373,96.592590,366.716019,366.216064,cat,3.136231,353.032872,0.583291,0.661110
414,000000198641.jpg,158.888062,181.367126,450.940247,473.419312,cat,0.802940,292.052185,1.000000,0.546914


In [9]:
_, bins = np.histogram(df['scale'], bins=3)
bins

array([0.        , 0.41639662, 0.83279324, 1.24918986])

In [125]:
# get anchors
anchor_parameters = AnchorParameters(
    sizes=[133, 267, 400, 534, 667],
    strides=[8, 16, 32, 64, 128],
    # ratio=h/w
    ratios=np.array([1, 0.5], keras.backend.floatx()),
    scales=np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)], keras.backend.floatx()),
)
anchors = anchors_for_shape((512, 512), anchor_params=anchor_parameters)
anchors.shape, anchors

((49104, 4),
 array([[ -62.5     ,  -62.5     ,   70.5     ,   70.5     ],
        [ -90.045204,  -43.022602,   98.045204,   51.022602],
        [-129.      ,  -29.25    ,  137.      ,   37.25    ],
        ...,
        [ -81.398254,  -81.398254,  977.39825 ,  977.39825 ],
        [-300.6822  ,   73.658905, 1196.6823  ,  822.3411  ],
        [-610.7965  ,  183.30087 , 1506.7965  ,  712.6991  ]],
       dtype=float32))

In [126]:
overlaps = compute_overlap(anchors.astype(np.float64), ann.astype(np.float64))

In [127]:
overlaps.shape

(49104, 10542)

In [128]:
argmax_overlaps_inds = np.argmax(overlaps, axis=1)
max_overlaps = overlaps[np.arange(overlaps.shape[0]), argmax_overlaps_inds]

In [129]:
filtered_anchors = anchors[max_overlaps >= 0.5]

In [130]:
len(filtered_anchors)/len(anchors)*100

98.59074617139133